# Dashboard con contatos electronicos del SECOP II

Grupo Jairo Ávila - Juan Gómez - Fabio Fonseca - Las gráficas y dashboards que se desarrollaran explicaran el comportamiento en los diferentes años de los valores de contratos para los departamentos de la costa caribe colombiana.

 1.) Instalación y cargue de librerias necesarias.

In [1]:

!pip install sodapy
!pip install python-dotenv
!pip install dash

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sodapy import Socrata
from dotenv import load_dotenv
load_dotenv()


import dash
import dash_core_components as dcc
import dash_html_components as html
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go 



C:\Users\ASUS\AppData\Local\Temp\ipykernel_45140\3710829311.py:15: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ASUS\AppData\Local\Temp\ipykernel_45140\3710829311.py:16: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


2.) Proceso de autenticación y cargar base de datos para el desarrollo del dash

In [2]:
#Autenticación
app_token = os.getenv("app_token")

client = Socrata("www.datos.gov.co",app_token)

# Cargar la base de datos del SECOP II
client = Socrata("www.datos.gov.co", None,timeout=200000)

### Descargar la data
### Se dejó como Limite una muestra de máximo 299,999 datos con el fin de un proceso de carga más rápido.
query="""
SELECT departamento as Departamento,
         nombre_entidad as Entidad,
            nit_entidad as NIT,
            valor_del_contrato as Valor,
            objeto_del_contrato as Objeto,
            tipo_de_contrato as Tipo,
            ciudad as Municipio,
            fecha_de_firma as Fecha


LIMIT 300000
"""

results = client.get("jbjy-vk9h", query=query)

# Códigos de clase

3.) Ajustes de la base de datos

In [3]:
# Creación y disposición de la base

Data = pd.DataFrame(results)
Data

,Departamento,Entidad,NIT,Valor,Objeto,Tipo,Municipio,Fecha
0,"San Andrés, Providencia y Santa Catalina",SERVICIO NACIONAL DE APRENDIZAJE SENA REGIONAL...,899999034,7149375,PRESTACIÓN DE SERVICIOS PERSONALES DE CARÁCTER...,Prestación de servicios,No Definido,2022-09-20T11:09:05.000
1,Valle del Cauca,SANTIAGO DE CALI DISTRITO ESPECIAL SECRETARIA...,890399011,17347000,Prestar los servicios de apoyo a la gestión a ...,Prestación de servicios,Cali,2022-07-23T11:07:01.000
2,Distrito Capital de Bogotá,UNIDAD ADMINISTRATIVA ESPECIAL DE SERVICIOS PÚ...,900126860,68366667,SAL017 Prestar los servicios profesionales a l...,Prestación de servicios,Bogotá,2021-03-24T18:03:31.000
3,Santander,MUNICIPIO DE BUCARAMANGA,890201222,18265333,PRESTAR SERVICIOS PROFESIONALES ENCAMINADOS A ...,Prestación de servicios,Bucaramanga,2022-08-11T11:08:51.000
4,Antioquia,ALCALDÍA MUNICIPIO DE BELLO,890980112,70000000,CONTRATO INTERADMINISTRATIVO DE ADMINISTRACION...,Prestación de servicios,Bello,2021-05-18T17:05:53.000
...,...,...,...,...,...,...,...,...
299995,Atlántico,DISTRITO ESPECIAL INDUSTRIAL Y PORTUARIO DE BA...,890102018,9105000,LA PRESTACIÓN DE SERVICIOS DE APOYO A LA GESTI...,Prestación de servicios,Barranquilla,2021-04-20T21:04:18.000
299996,Atlántico,DEPARTAMENTO DEL ATLANTICO,890102006,36050000,PRESTACIÓN DE SERVICIOS DE APOYO A LA GESTIÓN ...,Prestación de servicios,Barranquilla,2022-01-11T09:01:34.000
299997,Valle del Cauca,SENA CENTRO LATINOAMERICANO DE ESPECIES MENOR...,899999034,19582442,Prestación de servicios profesionales de carác...,Prestación de servicios,Tuluá,2020-07-07T08:07:12.000
299998,Distrito Capital de Bogotá,SENA DIRECCIÓN GENERAL Dirección Jurídica,899999034,5337393913,ESTUDIOS DISEÑOS LICENCIAMIENTO Y CONSTRUCCIÓN...,Obra,Bogotá,2020-09-11T19:09:11.000


In [4]:
# Ajuste de fechas

Data['Fecha']=pd.to_datetime(Data['Fecha'])

mes_numero=[1,2,3,4,5,6,7,8,9,10,11,12]
M=['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre',
        'Octubre','Noviembre','Diciembre']
Data['Mes']=Data['Fecha'].dt.month.replace(mes_numero,M)

Para poder desglozar año a año la información que se quiere mostrar es necesario crear una columna que especifique estos periodos dado lo anterior se crea una columna "Año"

In [5]:
Data['Año']=Data['Fecha'].dt.year.fillna(0).astype(int)

Data

,Departamento,Entidad,NIT,Valor,Objeto,Tipo,Municipio,Fecha,Mes,Año
0,"San Andrés, Providencia y Santa Catalina",SERVICIO NACIONAL DE APRENDIZAJE SENA REGIONAL...,899999034,7149375,PRESTACIÓN DE SERVICIOS PERSONALES DE CARÁCTER...,Prestación de servicios,No Definido,2022-09-20 11:09:05,Septiembre,2022
1,Valle del Cauca,SANTIAGO DE CALI DISTRITO ESPECIAL SECRETARIA...,890399011,17347000,Prestar los servicios de apoyo a la gestión a ...,Prestación de servicios,Cali,2022-07-23 11:07:01,Julio,2022
2,Distrito Capital de Bogotá,UNIDAD ADMINISTRATIVA ESPECIAL DE SERVICIOS PÚ...,900126860,68366667,SAL017 Prestar los servicios profesionales a l...,Prestación de servicios,Bogotá,2021-03-24 18:03:31,Marzo,2021
3,Santander,MUNICIPIO DE BUCARAMANGA,890201222,18265333,PRESTAR SERVICIOS PROFESIONALES ENCAMINADOS A ...,Prestación de servicios,Bucaramanga,2022-08-11 11:08:51,Agosto,2022
4,Antioquia,ALCALDÍA MUNICIPIO DE BELLO,890980112,70000000,CONTRATO INTERADMINISTRATIVO DE ADMINISTRACION...,Prestación de servicios,Bello,2021-05-18 17:05:53,Mayo,2021
...,...,...,...,...,...,...,...,...,...,...
299995,Atlántico,DISTRITO ESPECIAL INDUSTRIAL Y PORTUARIO DE BA...,890102018,9105000,LA PRESTACIÓN DE SERVICIOS DE APOYO A LA GESTI...,Prestación de servicios,Barranquilla,2021-04-20 21:04:18,Abril,2021
299996,Atlántico,DEPARTAMENTO DEL ATLANTICO,890102006,36050000,PRESTACIÓN DE SERVICIOS DE APOYO A LA GESTIÓN ...,Prestación de servicios,Barranquilla,2022-01-11 09:01:34,Enero,2022
299997,Valle del Cauca,SENA CENTRO LATINOAMERICANO DE ESPECIES MENOR...,899999034,19582442,Prestación de servicios profesionales de carác...,Prestación de servicios,Tuluá,2020-07-07 08:07:12,Julio,2020
299998,Distrito Capital de Bogotá,SENA DIRECCIÓN GENERAL Dirección Jurídica,899999034,5337393913,ESTUDIOS DISEÑOS LICENCIAMIENTO Y CONSTRUCCIÓN...,Obra,Bogotá,2020-09-11 19:09:11,Septiembre,2020


In [6]:
Data.dtypes

Departamento            object
Entidad                 object
NIT                     object
Valor                   object
Objeto                  object
Tipo                    object
Municipio               object
Fecha           datetime64[ns]
Mes                     object
Año                      int32
dtype: object

Otro proceso a realizar es convertir la variable Valor en float debido a que en el código anterior el tipo de esta variable es object.

In [7]:
Data['Valor'] = Data['Valor'].astype(float) 

In [8]:
Data.dtypes

Departamento            object
Entidad                 object
NIT                     object
Valor                  float64
Objeto                  object
Tipo                    object
Municipio               object
Fecha           datetime64[ns]
Mes                     object
Año                      int32
dtype: object

Al analizar el dataframe se encuentran diferentes años con el valor de 0 por lo que se procede a eliminar estos registros

In [9]:
Data = Data[Data['Año'] != 0]
Data

,Departamento,Entidad,NIT,Valor,Objeto,Tipo,Municipio,Fecha,Mes,Año
0,"San Andrés, Providencia y Santa Catalina",SERVICIO NACIONAL DE APRENDIZAJE SENA REGIONAL...,899999034,7.149375e+06,PRESTACIÓN DE SERVICIOS PERSONALES DE CARÁCTER...,Prestación de servicios,No Definido,2022-09-20 11:09:05,Septiembre,2022
1,Valle del Cauca,SANTIAGO DE CALI DISTRITO ESPECIAL SECRETARIA...,890399011,1.734700e+07,Prestar los servicios de apoyo a la gestión a ...,Prestación de servicios,Cali,2022-07-23 11:07:01,Julio,2022
2,Distrito Capital de Bogotá,UNIDAD ADMINISTRATIVA ESPECIAL DE SERVICIOS PÚ...,900126860,6.836667e+07,SAL017 Prestar los servicios profesionales a l...,Prestación de servicios,Bogotá,2021-03-24 18:03:31,Marzo,2021
3,Santander,MUNICIPIO DE BUCARAMANGA,890201222,1.826533e+07,PRESTAR SERVICIOS PROFESIONALES ENCAMINADOS A ...,Prestación de servicios,Bucaramanga,2022-08-11 11:08:51,Agosto,2022
4,Antioquia,ALCALDÍA MUNICIPIO DE BELLO,890980112,7.000000e+07,CONTRATO INTERADMINISTRATIVO DE ADMINISTRACION...,Prestación de servicios,Bello,2021-05-18 17:05:53,Mayo,2021
...,...,...,...,...,...,...,...,...,...,...
299995,Atlántico,DISTRITO ESPECIAL INDUSTRIAL Y PORTUARIO DE BA...,890102018,9.105000e+06,LA PRESTACIÓN DE SERVICIOS DE APOYO A LA GESTI...,Prestación de servicios,Barranquilla,2021-04-20 21:04:18,Abril,2021
299996,Atlántico,DEPARTAMENTO DEL ATLANTICO,890102006,3.605000e+07,PRESTACIÓN DE SERVICIOS DE APOYO A LA GESTIÓN ...,Prestación de servicios,Barranquilla,2022-01-11 09:01:34,Enero,2022
299997,Valle del Cauca,SENA CENTRO LATINOAMERICANO DE ESPECIES MENOR...,899999034,1.958244e+07,Prestación de servicios profesionales de carác...,Prestación de servicios,Tuluá,2020-07-07 08:07:12,Julio,2020
299998,Distrito Capital de Bogotá,SENA DIRECCIÓN GENERAL Dirección Jurídica,899999034,5.337394e+09,ESTUDIOS DISEÑOS LICENCIAMIENTO Y CONSTRUCCIÓN...,Obra,Bogotá,2020-09-11 19:09:11,Septiembre,2020


## 2.) Creación de dashboards

El primer dashaboard incluirá un análisis temporal del comportamiento del valor de los contratos en cada uno de los departamentos de la costa caribe colombiana. El proceso de desarrollo consta de los siguientes pasos:

El segundo gráfico en el dashboard está desarrollado para mostrar como es la proporción anual del valor de los contratos en cada año para los departamentos de la región Caribe en Colombia. Con esto tener una muestra de en cual departamento se destinaron los mayores contratos en cada año. 

In [10]:
# 1.) Inicialmente se filtran los datos a utilizar de los departamentos de la costa caribe colombiana
departamentos_costa = ['Atlántico', 'Bolívar', 'Cesar', 'Córdoba', 'La Guajira', 'Magdalena', 'Sucre']
Data_costa = Data[Data['Departamento'].isin(departamentos_costa)]

# 2.) Se agrupan los datos por Año y Departamento con un groupby, agregando el valor de colos contratos para que se sumen los valores de esta
Data_costa_Agrupada = Data_costa.groupby(['Año', 'Departamento'])['Valor'].sum().reset_index()

# 4.) Creación de la aplicación de Dash
app = dash.Dash(__name__)

# 3.) Generamos un gráfico de linea
fig_linea = px.line(Data_costa_Agrupada, x="Año", y="Valor", color="Departamento", title="Valor por Departamento en la Costa Caribe")

# 2.) Se crea la lista de los años que se encuentran en la base de datos de Costa
años = Data_costa['Año'].unique()

# 3.) Graficar el share mediante un gráfico de pastel para el año cero (inicial)

# Año cero
año_cero = años[0]

# Agrupación de datos para determinar la proporción anual
share_departamento = Data_costa[Data_costa['Año'] == año_cero].groupby('Departamento')['Valor'].sum() / Data_costa[Data_costa['Año'] == año_cero]['Valor'].sum()

#Gráfico de pastel
fig_pie = px.pie(share_departamento, names=share_departamento.index, values=share_departamento.values, 
             title=f"Share del valor de contratos por Departamento en {año_cero}")


# 5.) Definición de la interfaz de usuario de la aplicación
app.layout = html.Div([
    dcc.Dropdown(
        id='año-dropdown',
        options=[{'label': año, 'value': año} for año in años],
        value=año_cero
    ),
    dcc.Graph(id='pie-chart', figure=fig_pie),
    dcc.Graph(id='linea',figure=fig_linea)
])

# 6.) Definición del callback para actualizar el gráfico en función de cada año que se seleccione
@app.callback(
    Output('pie-chart', 'figure'),Output('linea', 'figure'),
    Input('año-dropdown', 'value')
)
def update_pie_chart(selected_año):
    share_por_departamento = Data_costa[Data_costa['Año'] == selected_año].groupby('Departamento')['Valor'].sum() / Data_costa[Data_costa['Año'] == selected_año]['Valor'].sum()
    fig_pie = px.pie(share_por_departamento, names=share_por_departamento.index, values=share_por_departamento.values, title=f"Share de Valor por Departamento en {selected_año}")
    
    return fig_pie,fig_linea




if __name__ == '__main__': app.run_server(debug=True)



De los resultados encontrados en los gráficos anteriores, podemos ver que desde el año 2019 el departamento del atlático tiene un repunte en con el total de los valores de los contratos asignados para dicho departamento, siendo el año 2022 donde se encuentra el mayor pico con valores cercanos a los 500 billones de pesos, agrupando cerca del 43% aproximadamente de los valores de los contratos de la region caribe. Desde el año 2020, diferente al atlático los departamentos con mayores valores en sus contratos son Bolivar y Cesar, teniendo este último una disminución para el año 2023 del total de estos valores en contratos.

Adicionalmente, se encunetra que en el año 2019 el departamento del bolivar, fue el destino del 92% de los contratos del norte de Colombia. 

